### 导入数据，进行初步的EDA。

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns



df = pd.read_csv('../data/Label/Updated_Medal_Counts_with_Athlete_Numbers.csv')

### 进行特征筛选与特征降维
- 首先分析变量与目标变量之间的相关性，然后分析变量与目标变量的互信息，综合考虑两者的结果，进行特征选择。
- 观察看有没有必要进行特征降维，如果有，可以考虑使用PCA等方法进行特征降维。

In [30]:
df.head()
target = ['Gold', 'Silver', 'Bronze','Total']
features = ['Rank', 'Host','Number_of_people']

# # 相关性分析
# corr = df[features+target].corr()
# plt.figure(figsize=(10, 10))
# sns.heatmap(corr, annot=True, cmap='coolwarm')
# plt.title('Correlation Matrix')
# plt.show()

    
# # 统计之前从来没有获得过奖牌的国家
medal_0_country = []
# for country in df['NOC'].unique():
#     country_df = df[df['NOC'] == country]
#     if country_df['Goden'].sum() == 0:
#         medal_0_country.append(country)
        
# print('The number of countries that have never won a medal:', len(medal_0_country))


df1 = pd.read_csv('../data/summerOly_athletes.csv')
#把Medal 这列的数据进行lable编码
df1['Medal'] = df1['Medal'].map({'Gold': 3, 'Silver': 2, 'Bronze': 1, 'No medal': 0})
for country in df1['NOC'].unique():
    country_df = df1[df1['NOC'] == country]
    #  观察Medal列，如果这个国家一直都是"no medal"，那么这个国家就是从来没有获得过奖牌的国家
    if country_df['Medal'].sum() == 0:
        medal_0_country.append(country)

        
        
print('The number of countries that have never won a medal:', len(medal_0_country))
print(medal_0_country)

    
    

The number of countries that have never won a medal: 77
['CHA', 'NCA', 'LBA', 'PLE', 'COM', 'BRU', 'MDV', 'YAR', 'CGO', 'BEN', 'SOM', 'MLI', 'ANG', 'BAN', 'ESA', 'HON', 'SEY', 'MTN', 'SKN', 'VIN', 'LBR', 'NEP', 'PLW', 'ASA', 'SAM', 'RWA', 'MLT', 'GUI', 'BIZ', 'YMD', 'SLE', 'PNG', 'YEM', 'OMA', 'VAN', 'IVB', 'CAF', 'MAD', 'MAL', 'BIH', 'GUM', 'CAY', 'GBS', 'TLS', 'COD', 'LAO', 'ROT', 'CAM', 'SOL', 'CRT', 'GEQ', 'BOL', 'SAA', 'ANT', 'AND', 'FSM', 'MYA', 'MAW', 'RHO', 'STP', 'LIE', 'GAM', 'COK', 'SWZ', 'NBO', 'ARU', 'NRU', 'VNM', 'BHU', 'MHL', 'KIR', 'UNK', 'TUV', 'NFL', 'SSD', 'LES', 'LBN']


### 接下来完成第一步也就是零膨胀模型的构建
- 首先判断历史数据的平稳性，使用Augmented Dickey-Fuller检验验证奖牌数时间序列的平稳性。
- 然后为第一部分（零膨胀逻辑回归）选择合适的特征，使用逻辑回归模型预测某国是否属于“永远无法获奖”的群体（结构性零值）。
- 对于第二部分（计数部分），如果数据不平稳就选择负二项分布，如果数据平稳就选择泊松分布。
- 最后，将两部分的结果相乘，得到最终的预测结果。


In [166]:
import statsmodels.api as sm
from statsmodels.discrete.count_model import ZeroInflatedNegativeBinomialP

# Load the data
df_score = pd.read_excel('E:/python/2025-MCM-ICM/data/Final_csv/result2.xlsx')
#去除"Unnamed"列
# df_score = df_score.loc[:, ~df_score.columns.str.contains('^Unnamed','Rank')]
# print(df_score.head())

train_df = df_score[df_score['Year'] < 2024]




#定义模型特征，对于零膨胀特征，我们使用Never_won_medal,Total_appearance,'Athletes_growth_rate', 'Total_appearance' 'Lag_athletes', 'Lag_events'Last_appearance_athletes', 'Last_appearance_events
# Select features with less correlation
# Define features for inflation and count models
infl_features  = ['Total_appearance','Lag_athletes']
target = 'Medal_Count'
count_features = ['People_Count','Last_appearance_athletes','Lag_athletes']

# Remove duplicate features between infl_features and count_features
count_features = [x for x in count_features if x not in infl_features]

# Check correlations
correlation_matrix = train_df[infl_features + count_features].corr()
# print("Feature correlations:\n", correlation_matrix)

# Find highly correlated feature pairs (above 0.7)
high_correlation = correlation_matrix.abs() > 0.7
# Get upper triangle of correlations
high_correlation = high_correlation.where(np.triu(np.ones(high_correlation.shape), k=1).astype(bool))
# Find feature pairs where correlation is > 0.7
high_corr_pairs = np.where(high_correlation)
high_corr_values = [(correlation_matrix.index[i], correlation_matrix.columns[j], correlation_matrix.iloc[i, j]) 
                    for i, j in zip(*high_corr_pairs) if correlation_matrix.iloc[i, j] > 0.7]
# print("\nHighly correlated feature pairs:")
# for feat1, feat2, corr in high_corr_values:
#     print(f"{feat1} - {feat2}: {corr:.3f}")
# print("Highly correlated feature pairs:\n", high_correlation)




# Create separate scalers for inflation and count features
from sklearn.preprocessing import StandardScaler
infl_scaler = StandardScaler()
count_scaler = StandardScaler()

# Prepare training data
train_infl = train_df[infl_features].copy()
train_count = train_df[count_features].copy()

# Standardize features
train_infl_scaled = infl_scaler.fit_transform(train_infl)
train_count_scaled = count_scaler.fit_transform(train_count)

# Convert back to DataFrame
train_infl_scaled = pd.DataFrame(train_infl_scaled, columns=infl_features, index=train_infl.index)
train_count_scaled = pd.DataFrame(train_count_scaled, columns=count_features, index=train_count.index)

train_y = train_df[target]

# Add constant terms for training data
train_count_scaled = pd.DataFrame(train_count_scaled, columns=count_features)
train_infl_scaled = pd.DataFrame(train_infl_scaled, columns=infl_features)
train_count_scaled = sm.add_constant(train_count_scaled)
train_infl_scaled = sm.add_constant(train_infl_scaled)

# Train ZINB model with standardized features
zinb = ZeroInflatedNegativeBinomialP(
    train_y,
    train_count_scaled,
    exog_infl=train_infl_scaled,
    inflation='logit'
)

# Use robust optimization method
result = zinb.fit(maxiter=10000, method='bfgs', disp=0)
print(result.summary())

#准备预测数据，使用2024年的数据
test_df = df_score[df_score['Year'] == 2024]
test_infl = test_df[infl_features].copy()
test_count = test_df[count_features].copy()

# Scale the test data using the corresponding scalers
test_count_scaled = pd.DataFrame(
    count_scaler.transform(test_count),
    columns=count_features,
    index=test_count.index
)
test_infl_scaled = pd.DataFrame(
    infl_scaler.transform(test_infl),
    columns=infl_features,
    index=test_infl.index
)


# Add constant term to test data
test_count_scaled = sm.add_constant(test_count_scaled)
test_infl_scaled = sm.add_constant(test_infl_scaled)

# Ensure columns are in the same order as training data
test_count_scaled = test_count_scaled[train_count_scaled.columns]
test_infl_scaled = test_infl_scaled[train_infl_scaled.columns]


#检查test_count_scaled和train_count_scaled的形状




# Calculate predictions
predicted_counts = result.predict(exog=test_count_scaled, exog_infl=test_infl_scaled)

# Calculate probability of zero medals using predict method
prob_zero = result.predict(exog=test_count_scaled, exog_infl=test_infl_scaled, which='prob-zero')

prob_at_least_one = 1 - prob_zero
# 将prob_at_least_one转换为DataFrame

prob_at_least_one = pd.Series(prob_at_least_one, index=test_df.index)
final_country = test_df[test_df['Never_won_medal'] == 0]['Team']

# 将prob_at_least_one通过final_country进行筛选，他们的index是一样的
prob_at_least_one = prob_at_least_one[final_country.index]
# print(prob_at_least_one)





# 最后，筛选出大于阈值的国家,并且这个国家从来没有获得过奖牌
prediction_df = pd.DataFrame({
    'Team': test_df['Team'],
    'Prob_at_least_one': prob_at_least_one,
    'Never_won_medal': test_df['Never_won_medal']
})


prediction_df = prediction_df[
    # (prediction_df['Prob_at_least_one'] > threshold) & 
    (prediction_df['Never_won_medal'] == 0)
]

prediction_df = prediction_df.sort_values('Prob_at_least_one', ascending=False)



## 使用蒙特卡洛模拟
n_simulations = 10000
simulate_count = []


for _ in range(n_simulations):
    # 对于每一个国家，我们都要进行一次模拟，生成伯努利随机变量（是否获奖）
    simulate_medal = np.random.binomial(1, prob_at_least_one)
    total_new_medal = simulate_medal.sum()
    simulate_count.append(total_new_medal)
    
simulate_count = np.array(simulate_count)

# 计算最可能值
value, counts = np.unique(simulate_count, return_counts=True)
most_likely_value = value[np.argmax(counts)]

# 计算置信区间
lower_bound = np.percentile(simulate_count, 2.5)
upper_bound = np.percentile(simulate_count, 97.5)


# 计算每个国家的预测命中频率
country_simulate_hits = {
    country: 0 for country in final_country
}
n_valid_simulations = 0

for _ in range(n_simulations):
    simulate_medal = np.random.binomial(1, prob_at_least_one)
    total_new_medal = simulate_medal.sum()
    if total_new_medal > 0:
        n_valid_simulations += 1
        for i, country in enumerate(final_country):
            if simulate_medal[i] == 1:
                country_simulate_hits[country] += 1
            

# 计算每个国家的预测命中频率
country_simulate_hit_rate = {
    country: hits / n_valid_simulations for country, hits in country_simulate_hits.items()
}

print(country_simulate_hit_rate)

# print("预测命中频率：")
# 筛选出预测命中频率最高的国家
predicted_countries = [country for country, rate in country_simulate_hit_rate.items() if rate > 0.2]
print(f"最可能首次获奖国家数量：{most_likely_value}")
print(f"97.5% 置信区间：[{lower_bound:.0f}, {lower_bound:.0f}]")
print("预测可能命中国家：", predicted_countries)


                     ZeroInflatedNegativeBinomialP Regression Results                    
Dep. Variable:                       Medal_Count   No. Observations:                 3016
Model:             ZeroInflatedNegativeBinomialP   Df Residuals:                     3012
Method:                                      MLE   Df Model:                            3
Date:                             周日, 26 1月 2025   Pseudo R-squ.:                  0.1605
Time:                                   02:46:42   Log-Likelihood:                -5401.7
converged:                                  True   LL-Null:                       -6434.2
Covariance Type:                       nonrobust   LLR p-value:                     0.000
                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
inflate_const               -1.6565      0.260     -6.379      0.000      -2.165      -1.147
i

In [79]:
from statsmodels.tsa.stattools import adfuller
from statsmodels.discrete.count_model import ZeroInflatedNegativeBinomialP


def check_stationarity(df):
    for col in target:
        # ADF test
        result = adfuller(df[col])
        print(f'ADF Statistic for {col}: {result[0]}')
        print(f'p-value: {result[1]}')
        if result[1] > 0.05:
            return False
    return True

## 开始训练零膨胀负二项模型


# 首先检查时间序列的平稳性
if check_stationarity(df[target]):
    print('Time Series is Stationary')

# 划分训练集和测试集
from sklearn.model_selection import train_test_split
X = df[features]
y = df[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
formular = f'{target[0]} ~ {" + ".join(features)}'

# 训练模型
model = ZeroInflatedNegativeBinomialP.from_formula(formular, data=X_train,exog_infl=~1)
zinb_results = model.fit(maxiter=1000)

## 预测
y_pred = zinb_results.predict(X_test,which='prob')

#计算ROC曲线
from sklearn.metrics import roc_curve, roc_auc_score
fpr, tpr, thresholds = roc_curve(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)

plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)

#寻找最佳阈值
optimal_idx = np.argmax(tpr - fpr)
optimal_threshold = thresholds[optimal_idx]
plt.scatter(fpr[optimal_idx], tpr[optimal_idx], color='red', label='Best Threshold')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic Curve')



#预测下一届奥运会哪些国家第一次获得奖牌

next_year = 2028
prediction = zinb_results.predict(df[features.keys()])
print(f'Predicted Medal Counts for {next_year} Olympics:')
print(prediction)







        

KeyError: 'Medal_Count'

### 第二阶段使用混合预测模型
- 首先使用ARIMAX进行时间序预测，结合历史数据和外部因素，预测下一届奥运会的奖牌数。
- 然后结合Xgboost进行残差预测，得到最终的预测结果。

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pmdarima import auto_arima
from sklearn.preprocessing import StandardScaler

## 平稳性处理模块
def make_stationary(series):
    d = 0 
    while adfuller(series)[1] > 0.05:
        series = series.diff().dropna()
        d += 1
    return series, d

#从外部变量预处理模块
# def prepare